In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import psycopg2
import requests
from openai import OpenAI
import numpy as np

/Users/jeongwon/Desktop/Chatbot/Law/lawchat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#Sentence similarity모델을 API로 하기 위한 함수.
def query(src, other_sentence, headers, Embedding_URL):
    payload = { 
        "inputs": {
        "source_sentence": src,
	    "sentences": other_sentence
        },
    }
    response = requests.post(Embedding_URL, headers=headers, json=payload)
    return response.json()
	
def find_similar_sentence(req, top_k):    
    # PostgreSQL 연결
    query_embeddings = embed_model.get_text_embedding(req)

    # SQL 실행
    sql_query = """
    WITH user_query AS (
        SELECT ARRAY {query_embeddings}::VECTOR(768) AS query_embedding
    )
    SELECT 
        name, part_num, part_name, chap_num, char_name, sec_num, sec_name, para_num, para_name, art_num, art_name, content,
        embedding <=> user_query.query_embedding AS distance
    FROM 
        each_paragraph, user_query
    ORDER BY 
        distance ASC
    LIMIT {top_k};
    """.format(query_embeddings=query_embeddings, top_k = top_k)
    cursor.execute(sql_query, query_embeddings)
    rows = cursor.fetchall()

    candidate = []
    for row in rows:
        row = row[:12]
        filtered_tuple = tuple(element for element in row if element is not None) #None값을 필터링하는 코드
        filtered_text = ' '.join(filtered_tuple)
        candidate.append(filtered_text)
    return candidate

#유사한 문장들끼리 유사성 검증, 유사한 문장들끼리의 평균 값 이상만 사용하도록
def extract_goodContext(candidate, top_k, headers, Embedding_URL):
    embed = []
    for one in candidate:
        while(1): #아주 가끔 API가 오작동할 때가 있기에 이를 보장해주기 위한 반복문
            try:
                strings = query(one, candidate, headers, Embedding_URL)
                embed.append(list(map(float, strings)))
                break
            except:
                print(f"String: {strings}")
                continue
    ave = 0
    for i in range(len(embed)):
        embed[i] = np.mean(embed[i])
        ave += embed[i]
    avg_embed = ave/len(embed)
    index_candidate = [i for i in range(len(embed)) if embed[i] > avg_embed]

    elected_candidate = [candidate[i] for i in index_candidate]
    context = "\n".join(elected_candidate)
    return context


In [ ]:
embed_model = HuggingFaceEmbedding(model_name="jhgan/ko-sroberta-multitask")
conn = psycopg2.connect(host='host', dbname='dbname', user='user', password='password', port=5432)#you need to add your information
cursor = conn.cursor()

OpenAI_key = 'OpenAI_key'
Embedding_URL = "https://api-inference.huggingface.co/models/jhgan/ko-sroberta-multitask"
headers = {"Authorization": "HuggingFace API Key"}

In [ ]:
template = """
아래의 템플릿에 맞춰서 답변해줘.

질문:{request}

관련 법령 정리: 여기에 제공한 맥락에 나와있는 법률들을 번호를 매겨 자세히 정리해줘.

답변: 여기에 질문에 관한 자세한 답변을 해줘.
"""
top_k = 10
client = OpenAI(api_key=OpenAI_key)

In [ ]:
while(1):
    #fp = open('result.txt','a')
    request = input("Question: ")
    candidate = find_similar_sentence(request, top_k)

    #유사한 문장들끼리 유사성 검증, 유사한 문장들끼리의 평균 값 이상만 사용하도록
    context = extract_goodContext(candidate, top_k, headers, Embedding_URL)

    #LLM에게 질문하는 파트
    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": f"당신은 법의 세부적인 내용을 모르는 일반인들에게 질문을 받고 답을 제공하는 전문가입니다. 이어지는 맥락을 참고하여 자세하게 답변 해주세요. 그리고 반드시 법의 출처를 밝혀주세요. \n 맥락:{context}" },
        {"role": "user", "content": template.format(request=request)}
    ],
    )
    
    response = completion.choices[0].message.content
    print(response)
    print()
    #fp.write(response+'\n-----------------------------------------------\n')
    #fp.close()

질문: 중대재해처벌법에서 말하는 사고의 범위는 어떻게 되나요?

관련 법령 정리: 

1. 중대재해처벌법 제1장 총칙 제2조 (정의)
2. 산업안전보건법 제1장 총칙 제2조 (정의)
3. 산업안전보건법시행령 제10장 보칙 제110조 (제재 요청 대상 등)
4. 산업안전보건법시행규칙 제1장 총칙 제3조 (중대재해의 범위)

답변: 중대재해처벌법에서 정의하는 "중대재해"는 크게 "중대산업재해"와 "중대시민재해"로 구분됩니다.

1. **중대산업재해**: 이는 산업안전보건법에 따른 산업재해 중 특정 결과를 초래한 경우로 정의됩니다. 여기에는 다음과 같은 사고가 포함됩니다:
   - 사망자가 1명 이상 발생한 경우.
   - 동일한 사고로 인해 6개월 이상의 치료가 필요한 부상자가 2명 이상 발생한 경우.
   - 동일한 유해 요인으로 인해 급성중독 등 특정 직업성 질병자가 1년 이내에 3명 이상 발생한 경우.

2. **중대시민재해**: 이는 특정 원료, 제조물이나 공중이용시설, 공중교통수단의 결함으로 인한 재해를 말하며, 결과적으로 다음 조건 중 하나를 충족합니다. 단, 중대산업재해로 분류되는 경우는 제외됩니다: 
   - 사망자가 1명 이상 발생한 경우.
   - 동일한 사고로 2개월 이상의 치료가 필요한 부상자가 10명 이상 발생한 경우.
   - 동일한 원인으로 인해 3개월 이상 치료가 필요한 질병자가 10명 이상 발생한 경우.

요약하자면, 중대재해처벌법은 특정 위험 상황으로 인한 중대한 결과를 초래한 재해를 "중대재해"로 정의하고 있으며, 이는 발생한 사고의 심각도와 피해자 수를 기준으로 측정됩니다. 이 법은 사업주 및 경영책임자가 책임을 지고 재해 예방에 필요한 조치를 취하도록 규정하고 있으며, 산업현장과 공공시설 등 다양한 영역에 걸쳐 적용됩니다.

질문: 산업안전보건 시행규칙의 정의는 무엇인가요?

관련 법령 정리: 
1. 산업안전보건법시행규칙 제4장 유해ㆍ위험 방지 조치 제63조 (기계ㆍ설비 등에 대한 안전 및 보건조치)
2. 산업안전보건법시행규칙